In [0]:
import io
from google.colab import drive
drive.mount('/content/drive')


In [0]:
f = open("/content/drive/My Drive/Dataset/Veales Dream Symbols.csv", encoding="utf8")
lines = f.readlines()
f.close()
lines = [l.strip() for l in lines]

In [0]:
#get the unique headers
headers = lines[0].split(",")
headers = [h for h in headers if h != '']
headers = set(headers)

#remove the headers
lines = lines[1:]

#get the symbols
symbols = []

for line in lines:
    line = line.split(",")
    symbols.append(line[0])

In [0]:
#get all the types in the data
types = []
type_instances = []
for line in lines:
    words = line.split(",")
    w_type = words[1]
    types.append(w_type)
type_instances= types
unique = set(types)
types = [t for t in unique]
bug = types[17]
types = [t for t in types if t != bug]

#cleaning the dream type area
for i in range(len(type_instances)):
    line = lines[i]
    for t in types:
        if t in line:
            type_instances[i] = t
            
#create origin rule
origin_rule = '"origin":['
main_rule = ''
for i in range(len(symbols)):
    if i == len(symbols) - 1:
        origin_rule += '"#' + symbols[i] + '#"]'
        main_rule += '"' + symbols[i] + '":["Card drawn:' + \
    symbols[i] + ', #' + type_instances[i] + '#, #' + \
    type_instances[i]+'#"]'
    else:
        origin_rule += '"#' + symbols[i] + '#",' 
        main_rule += '"' + symbols[i] + '":["Card drawn:' + \
    symbols[i] + ', #' + type_instances[i] + '#, #' + \
    type_instances[i]+'#"],'


In [0]:
#find all the meanings relating to cards in a type 
type_dict = {}
for t in types:
    type_res = []
    for line in lines:
        l = line.split(",")
        if l[1] == t:
            l = l[2:]
            line = [m for m in l if m !='']
            type_res.append(line)
    type_dict[t] = type_res

#clean 
#find cut off words to make meaning similar to taroe card vocab 
cut_off_word = "your dream"
c_size = len(cut_off_word)
#stores the remaining sentence after the cut off words
start_off_word = "you"

for key in type_dict:
    new_meaning = []
    meanings = type_dict[key]
    
    for i in range(len(meanings)):
        means = meanings[i]
        for i in range(len(means)):
            m = means[i]
            i = m.find(start_off_word)
            m = m[i:]
            
            #find your dream
            if m.find(cut_off_word) > -1:
                j = m.find(cut_off_word)
                m = m[j + c_size:]
            new_meaning.append(m)
        
    #append new meaning to symbol
    type_dict[key] = new_meaning

In [0]:
#clean
for key in type_dict:
    clean_m = []
    meanings = type_dict[key]
    
    for i in range(len(meanings)):
        clean1 = meanings[i].strip('.')
        clean2 = clean1.strip('"')
        clean_m.append(clean2)
    type_dict[key] = clean_m

for key in type_dict:
    meanings = []
    meanings = type_dict[key]
    
    while '' in meanings:
        meanings.remove('')
    type_dict[key] = meanings

#create rule for each types
meaning_rule = ""
for t in types:
    type_rule = '"' + t + '":['
    #get all the values for a type
    messages = type_dict[t]
    for msg in messages:
        type_rule += '"' + msg + '",'
    type_rule = type_rule[:-1]
    if t != 'Machine':
        type_rule += '],'
    else:
        type_rule +=']'
    meaning_rule += type_rule

#combine to form tracery grammar
tracery = '{' + origin_rule + ',' + main_rule + ',' + \
meaning_rule + '}'

#write to file
fout = open("/content/drive/My Drive/tarot.txt", "w")
fout.write(tracery)
fout.close()